# How to plot one large MOC (union) per service

## Import required modules

In [1]:
from elasticsearch import Elasticsearch
from mocpy import MOC
from ipyaladin import Aladin

## Connect to the ElasticSearch server

In [2]:
es = Elasticsearch('http://voparis-elasticsearch.obspm.fr:9200')
indexName = 'moc-index'
target = "mars"

## Get the list of services

In [3]:
query = {
    "bool": {
         "must": [
             {
                "exists" : {
                    "field" : "coverage"
                }
             },
             {
                 "match" : {
                    "target_name" : target
                 }
            }
        ]
    }
}

aggs = {
    "agg-service" : {
        "terms": {
            "field" : "service_title.keyword",
            "size": 100
        },
    }
}


page = es.search(
    index=indexName,
    query=query,
    size=0,
    fields=["*"],
    aggs = aggs
)

services = []

for bucket in page["aggregations"]["agg-service"]["buckets"] :
    print(bucket["key"])
    services.append(bucket["key"])

Mars_craters_dev
CRISM
hst_planeto
hrsc3nd
Planmap
vvex


## For each service, merge all of its MOCs into one for the target

In [82]:
def mergeMocs(mocsList) :
    mocUnion = MOC.new_empty(29)
    for item in mocsList :
        coverageField = item["fields"]["coverage"]
        if type(coverageField) == type([]) :
            for coverage in coverageField :
                moc = MOC.from_str(coverage)
                mocUnion = mocUnion.union(moc)
        else :
            moc = MOC.from_str(coverageField)
            mocUnion = mocUnion.union(moc)
    return mocUnion

In [83]:
size = 1000

data = []
for service in services :
    print("Merging MOCs of ",service,"...")
    query = {
        "bool": {
             "must": [
                 {
                    "exists" : {
                        "field" : "coverage"
                    }
                },
                 {
                     "match" : {
                        "target_name" : target
                     }
                },
                {
                    "term" : {
                         "service_title.keyword" : service
                     } 
                 } 
            ]
        }
    }

    page = es.search(
        index=indexName,
        query=query,
        size=size,
        scroll="2m",
        fields=["coverage"],
        aggs = aggs
    )
    
    sid = page['_scroll_id']
    scroll_size = page['hits']['total']["value"]
    mocUnion = MOC.new_empty(29)

    progress = 0
    increment = (100*size)/page['hits']['total']["value"]
    
    while (scroll_size > 0):
        print("%.2f %%" % progress)
        page = es.scroll(scroll_id = sid, scroll = '2m')
        # Update the scroll ID
        sid = page['_scroll_id']
        # Get the number of results that we returned in the last scroll
        scroll_size = len(page['hits']['hits'])
        mocUnion = mocUnion.union(mergeMocs(page["hits"]["hits"]))
        progress = progress + increment
        
    data.append({"service":service,"coverage":mocUnion})

priont("DONE")

Merging MOCs of  Mars_craters_dev ...
0.00 %
0.26 %
0.52 %
0.78 %
1.04 %
1.30 %
1.56 %
1.82 %
2.08 %
2.34 %
2.60 %
2.86 %
3.12 %
3.38 %
3.64 %
3.90 %
4.16 %
4.42 %
4.68 %
4.94 %
5.20 %
5.46 %
5.72 %
5.98 %
6.24 %
6.50 %
6.76 %
7.02 %
7.28 %
7.54 %
7.80 %
8.06 %
8.32 %
8.58 %
8.84 %
9.10 %
9.36 %
9.62 %
9.88 %
10.14 %
10.40 %
10.66 %
10.92 %
11.18 %
11.44 %
11.70 %
11.96 %
12.22 %
12.48 %
12.74 %
13.00 %
13.26 %
13.52 %
13.78 %
14.04 %
14.30 %
14.56 %
14.82 %
15.08 %
15.34 %
15.60 %
15.86 %
16.12 %
16.38 %
16.64 %
16.90 %
17.16 %
17.42 %
17.68 %
17.94 %
18.20 %
18.46 %
18.72 %
18.98 %
19.24 %
19.50 %
19.76 %
20.02 %
20.28 %
20.54 %
20.80 %
21.06 %
21.32 %
21.58 %
21.84 %
22.10 %
22.36 %
22.62 %
22.88 %
23.14 %
23.40 %
23.66 %
23.92 %
24.18 %
24.44 %
24.70 %
24.96 %
25.22 %
25.48 %
25.74 %
26.00 %
26.26 %
26.52 %
26.78 %
27.04 %
27.30 %
27.56 %
27.82 %
28.08 %
28.34 %
28.60 %
28.86 %
29.12 %
29.38 %
29.64 %
29.90 %
30.16 %
30.42 %
30.68 %
30.94 %
31.20 %
31.46 %
31.72 %
31.98 %
32.24 %
3

In [84]:
aladin = Aladin(
    coo_frame='body',
    survey="http://voparis-srv-paris.obspm.fr/vo/planeto/hips/CDS_P_Mars_Viking-MDIM21-color/"
)
aladin

Aladin(coo_frame='body', options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_sur…

In [88]:
for mocService in data :
    jsonMoc = mocService["coverage"].serialize(format='json', optional_kw_dict=None, pre_v2=False)
    aladin.add_moc_from_dict(jsonMoc, {'opacity' : 0.5, 'name' : mocService["service"]})